In [1]:
%pip install tensorboard --quiet
%pip install seaborn --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys
sys.path.append('/workspace/cropClassification')
sys.path.append('/workspace/cropClassification/model')
import os
# if using Apple MPS, fall back to CPU for unsupported ops
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
import torch
import pandas as pd
from torch.utils.data import DataLoader

from unet import UNetWithAncillary
from compiler import ModelCompiler
from dataloader import RoadsideCropImageDataset
from loss import BalancedCrossEntropyLoss

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")  # Use GPU with CUDA
    print("Using CUDA")
elif torch.backends.mps.is_available():
    device = torch.device("mps")  # Use Apple M1/M2 GPU with MPS (Metal Performance Shaders)
    print("Using MPS")
else:
    device = torch.device("cpu")  # Fall back to CPU
    print("Using CPU")

Using CUDA


### Configuration

In [4]:
config = {
    "model": {
        "type": "UNetWithAncillary",  # Example model type, replace with your actual model class name
        "params": {
            "in_channels": 9,  # Since we are using 9-channel input images
            "out_channels": 3   # Number of output classes for segmentation
        }
    },
    "training": {
        "epochs": 50,
        "batch_size": 32,
        "learning_rate": 0.01,
        "optimizer": {
            "type": "Adam",
            "params": {
                "lr": 0.01
            }
        },
        "scheduler": {
            "type": "StepLR",
            "params": {
                "step_size": 10,
                "gamma": 0.5
            }
        },
        "criterion": BalancedCrossEntropyLoss(),  # or you can use your custom loss like BalancedCrossEntropyLoss
        "resume" : False,
        "resume_epoch" : None,
    },
    "validation": {
        "epochs": 50,
        "batch_size": 16
    },
    "dataset": {
        "train_csv": "/workspace/data/data/masked_data_csiss/training/train_chipping_csv_w_anc.csv",  # Path to the training DataFrame (includes npy file paths)
        "val_csv": "/workspace/data/data/masked_data_csiss/validation/validation_chipping_csv_w_anc.csv", # Path to the validation DataFrame (includes npy file paths)
        "train_root_path": "/workspace/data/data/masked_data_csiss/training",
        "val_root_path": "/workspace/data/data/masked_data_csiss/validation",
        "image_column": "img_chip_path",             # Column containing the image paths (npy files)
        "mask_column": "lbl_chip_path",              # Column containing the mask paths
        "train_mean": [93.8785585, 111.81092494, 76.94555781, 113.58929434, 206.93557473, 28.98472963,
                       46.53684269, 113.74437327, 116.23856585],  # Mean values for training set normalization
        "train_std": [53.24595916, 46.34658429, 45.91157286, 47.69937365, 82.32605363, 48.02506071,
                       22.46416468, 52.31732116, 47.89290138],   # Std values for training set normalization
        "val_mean": [88.62211239, 111.27628711, 75.19030815, 111.99622799, 223.21125121, 25.98495476, 
                     49.64829283, 115.91463906, 114.01573621],    # Mean values for validation set normalization
        "val_std": [53.97481266, 46.47043658, 45.64034871, 48.20284577, 59.89659002, 41.7467965,
                     20.82148233, 54.41768437, 47.68361442],     # Std values for validation set normalization
    },
    "evaluation": {
        "filename": "csiss_street_view_crop_classification.csv",  # Evaluation metrics to be used
        "class_mapping": {
            0: "Background",
            1: "Maize",
            2: "Soybean"
        }
    }
}


### Loading Datasets

In [5]:
train_df = pd.read_csv(config['dataset']['train_csv'])
val_df = pd.read_csv(config['dataset']['val_csv'])

In [6]:
train_dataset = RoadsideCropImageDataset(
    dataframe=train_df,
    root_dir=config['dataset']['train_root_path'],  # Root directory where images are stored
    usage='train',  # Indicates training dataset
    mean=config['dataset']['train_mean'],
    std=config['dataset']['train_std'],
    use_ancillary=True
)

val_dataset = RoadsideCropImageDataset(
    dataframe=val_df,
    root_dir=config['dataset']['val_root_path'],  # Root directory where images are stored
    usage='val',  # Indicates validation dataset
    mean=config['dataset']['val_mean'],
    std=config['dataset']['val_std'],
    use_ancillary=True
)

In [7]:
train_loader = DataLoader(train_dataset, batch_size=config['training']['batch_size'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config['training']['batch_size'], shuffle=False)

### Construct Model and Training

In [8]:
model = UNetWithAncillary(n_channels=config['model']['params']['in_channels'],
                          n_classes=config['model']['params']['out_channels'],
                          ancillary_data_dim=3)
model_comp = ModelCompiler(model=model,
                           params_init=None)

---------- GPU (CUDA) available ----------
Total number of trainable parameters: 14.8M
---------- UNetWithAncillary model compiled successfully ----------


In [9]:
model_comp.fit(
    trainDataset=train_loader,
    valDataset=val_loader,
    epochs=config['training']['epochs'],
    optimizer_name=config['training']['optimizer']['type'],
    lr_init=config['training']['learning_rate'],
    lr_policy='steplr',  # Use 'steplr' as specified in config
    criterion=config['training']['criterion'],
    log=True,  # Enable logging for TensorBoard
    return_loss=False,
    use_ancillary=True,
    **config['training']['scheduler']['params'])

-------------------------- Start training --------------------------
----------------------- [1/50] -----------------------
Epoch Training Loss: 0.6175
Current Learning Rate: 0.010000
Validation Loss: 0.5869
Epoch 1 completed in 158.61 seconds
----------------------- [2/50] -----------------------
Epoch Training Loss: 0.5590
Current Learning Rate: 0.010000
Validation Loss: 0.5823
Epoch 2 completed in 159.94 seconds
----------------------- [3/50] -----------------------
Epoch Training Loss: 0.5365
Current Learning Rate: 0.010000
Validation Loss: 0.5945
Epoch 3 completed in 163.80 seconds
----------------------- [4/50] -----------------------
Epoch Training Loss: 0.5057
Current Learning Rate: 0.010000
Validation Loss: 0.5077
Epoch 4 completed in 134.97 seconds
----------------------- [5/50] -----------------------
Epoch Training Loss: 0.4844
Current Learning Rate: 0.010000
Validation Loss: 0.6184
Epoch 5 completed in 142.51 seconds
----------------------- [6/50] -----------------------
E

In [8]:
model = UNetWithAncillary(n_channels=config['model']['params']['in_channels'],
                          n_classes=config['model']['params']['out_channels'],
                          ancillary_data_dim=3)
model_comp = ModelCompiler(model=model,
                           params_init="/workspace/notebook/outputs/UNetWithAncillary_ep50/chkpt/50_checkpoint.pth.tar")

---------- GPU (CUDA) available ----------
Total number of trainable parameters: 14.8M
---------- Pre-trained UNetWithAncillary model compiled successfully ----------


/workspace/cropClassification/model/compiler.py:70: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  params_init = torch.load(dir_params)


In [9]:
model_comp.accuracy_evaluation(eval_dataset=val_loader,
                               filename=config['evaluation']['filename'], 
                               num_classes=config['model']['params']['out_channels'], 
                               class_mapping=config['evaluation']['class_mapping'])

---------------- Start evaluation ----------------


RuntimeError: mat1 and mat2 must have the same dtype, but got Long and Float